In [3]:
import pandas as pd
import numpy as np
import geopandas as gpd
import os
dic="/data/MyDataBase/SWATGenXAppData/NHDPlusData"
streams=pd.read_pickle(os.path.join(dic,'streams.pk1'))

In [19]:
import arcpy
gdb_path = fr"D:\MyDataBase\NHDPlusData\NHDPLUS_H_0407_HU4_GDB\NHDPLUS_H_0407_HU4_GDB.gdb"

desc = arcpy.Describe(gdb_path)

# List all child datasets
tables_features =[]
for child in desc.children:
    print(child.name)
    if child.dataType == "FeatureDataset":
        for grandchild in child.children:
            #print(f"  {grandchild.name}")
            tables_features.append(grandchild.name)
            
print(tables_features)

ExternalCrosswalk
NHDFCode
NHDFeatureToMetadata
NHDMetadata
NHDPlusDivFracMP
NHDPlusEROMMA
NHDPlusEROMQAMA
NHDPlusEROMQARPT
NHDPlusFlow
NHDPlusFlowlineVAA
NHDPlusIncrLat
NHDPlusIncrPrecipMA
NHDPlusIncrPrecipMM01
NHDPlusIncrPrecipMM02
NHDPlusIncrPrecipMM03
NHDPlusIncrPrecipMM04
NHDPlusIncrPrecipMM05
NHDPlusIncrPrecipMM06
NHDPlusIncrPrecipMM07
NHDPlusIncrPrecipMM08
NHDPlusIncrPrecipMM09
NHDPlusIncrPrecipMM10
NHDPlusIncrPrecipMM11
NHDPlusIncrPrecipMM12
NHDPlusIncrROMA
NHDPlusIncrTempMA
NHDPlusIncrTempMM01
NHDPlusIncrTempMM02
NHDPlusIncrTempMM03
NHDPlusIncrTempMM04
NHDPlusIncrTempMM05
NHDPlusIncrTempMM06
NHDPlusIncrTempMM07
NHDPlusIncrTempMM08
NHDPlusIncrTempMM09
NHDPlusIncrTempMM10
NHDPlusIncrTempMM11
NHDPlusIncrTempMM12
NHDPlusMegaDiv
NHDPlusNHDPlusIDGridCode
NHDReachCodeMaintenance
NHDReachCrossReference
NHDSourceCitation
NHDVerticalRelationship
NHDPlus
Hydrography
NHDMetaToFeature
NHDMetadataHasSourceCitation
NHDPlusProcessingParameters
NHDProcessingParameters
ProcessingParameters
WBD


In [8]:
###### we first import Flowlines, catchments, waterbodies and attribute tables.
import pandas as pd
import numpy as np
import geopandas as gpd
import os
######################### DO NOT REMOVE THESE #####################################################
def egeolit(VPUID, layer_name,crs=26990):
    gdfs = []
    

    gdb_path = fr"D:\MyDataBase\NHDPlusData\NHDPLUS_H_0{VPUID}_HU4_GDB\NHDPLUS_H_0{VPUID}_HU4_GDB.gdb"
    
    data = gpd.read_file(gdb_path, layer=layer_name)
    try: 
        data=data.to_crs(f'EPSG:{crs}')
    except:
        print(f'No geometry for {layer_name}')
        data=data.drop(columns='geometry')
        
    if 'WBD' in layer_name:
        data.columns = data.columns.str.lower()
        data['VPUID']=VPUID
    if 'FDate' in data.columns:
        data=data.drop(columns='FDate')
    if 'Enabled' in data.columns:
        data=data.drop(columns='Enabled')
    if 'NHDPlusID' in data.columns:
        data['NHDPlusID']=data['NHDPlusID'].astype('int64')
        
        
    for i in ['NHDPlusID','ToNode','FromNode','HydroSeq','UpHydroSeq', 'DnHydroSeq','FromNHDPID','ToNHDPID','NodeNumber','huc12']:
        
            
        if i in data.columns:
            if data[i].isnull().any()==False:
                data[i]=data[i].astype('int64')
            else:
                print('THis layer has nan values',i)
    
    uselesscol=['Resolution', 'GNIS_ID', 'GNIS_Name','Shape_Length','ElevFixed','MaxElevRaw','RtnDiv','ToMeas','LevelPathI','InNetwork','InNetwork','ReachCode','VisibilityFilter','Elevation','Shape_Area','GapDistKm','DnLevelPat','DnLevel','Thinner','VPUIn', 'VPUOut','DivDASqKm','StatusFlag','FlowDir','loaddate']
    for i in uselesscol:
        if i in data.columns:
            data = data.drop(columns=i)
        
    return data 

import pandas as pd
import concurrent.futures

dic = fr"/data/MyDataBase/SWATGenXAppData/"

def fetch_data(args):
    VPUID, data_type, crs = args
    return egeolit(VPUID, data_type, crs=crs)  ### goeelit is a function to extract info from NHDPlus HR

def fetch_and_save_data(VPUIDs, data_types, base_path, crs=26990):
    # Generate all combinations of VPUIDs and data_types
    tasks = [(vpu, data_type, crs) for vpu in VPUIDs for data_type in data_types]
    
    with concurrent.futures.ThreadPoolExecutor() as executor:
        data_c = list(executor.map(fetch_data, tasks))
    
    # Split and save by data_type
    for data_type in data_types:
        filtered_data = [data for data, task in zip(data_c, tasks) if task[1] == data_type]
        combined_data = pd.concat(filtered_data)
        combined_data.to_pickle(os.path.join(base_path , f'NHDPlusData/{data_type}.pk1'))

VPUIDs = [405, 406, 407, 408, 409, 410]
data_types = ['NHDPlusIncrPrecipMA','NHDPlusEROMMA','NHDPlusIncrTempMA']
fetch_and_save_data(VPUIDs, data_types, dic)

No geometry for NHDPlusIncrPrecipMA
No geometry for NHDPlusIncrTempMA
No geometry for NHDPlusIncrTempMA
No geometry for NHDPlusIncrPrecipMA
No geometry for NHDPlusIncrPrecipMA
No geometry for NHDPlusIncrPrecipMA
No geometry for NHDPlusIncrPrecipMA
No geometry for NHDPlusIncrTempMA
No geometry for NHDPlusIncrTempMA
No geometry for NHDPlusIncrPrecipMA
No geometry for NHDPlusIncrTempMA
No geometry for NHDPlusIncrTempMA
No geometry for NHDPlusEROMMA
No geometry for NHDPlusEROMMA
No geometry for NHDPlusEROMMA
No geometry for NHDPlusEROMMA
No geometry for NHDPlusEROMMA
No geometry for NHDPlusEROMMA


In [11]:
import glob
glob.glob(fr"/data/MyDataBase/SWATGenXAppData/NHDPlusData/" +'*.pk1')

['/data/MyDataBase/SWATGenXAppData/NHDPlusData\\catchments.pk1',
 '/data/MyDataBase/SWATGenXAppData/NHDPlusData\\Flowlines.pk1',
 '/data/MyDataBase/SWATGenXAppData/NHDPlusData\\NHDFlowline.pk1',
 '/data/MyDataBase/SWATGenXAppData/NHDPlusData\\NHDPlusCatchment.pk1',
 '/data/MyDataBase/SWATGenXAppData/NHDPlusData\\NHDPlusEROMMA.pk1',
 '/data/MyDataBase/SWATGenXAppData/NHDPlusData\\NHDPlusFlowlineVAA.pk1',
 '/data/MyDataBase/SWATGenXAppData/NHDPlusData\\NHDPlusIncrPrecip.pk1',
 '/data/MyDataBase/SWATGenXAppData/NHDPlusData\\NHDPlusIncrPrecipMA.pk1',
 '/data/MyDataBase/SWATGenXAppData/NHDPlusData\\NHDPlusIncrTemp.pk1',
 '/data/MyDataBase/SWATGenXAppData/NHDPlusData\\NHDPlusIncrTempMA.pk1',
 '/data/MyDataBase/SWATGenXAppData/NHDPlusData\\NHDWaterbody.pk1',
 '/data/MyDataBase/SWATGenXAppData/NHDPlusData\\streams.pk1',
 '/data/MyDataBase/SWATGenXAppData/NHDPlusData\\subbasins.pk1',
 '/data/MyDataBase/SWATGenXAppData/NHDPlusData\\waterbodies.pk1',
 '/data/MyDataBase/SWATGenXAppData/NHDPlusData

In [15]:
dic="/data/MyDataBase/SWATGenXAppData/"
# (QAMA)= the mean annual streamflow values.
NHDPlusIncrPrecipMA = pd.read_pickle(os.path.join(dic,'NHDPlusData/NHDPlusIncrPrecipMA.pk1'))
NHDPlusIncrTempMA = pd.read_pickle(os.path.join(dic,'NHDPlusData/NHDPlusIncrTempMA.pk1'))
NHDPlusEROMMA = pd.read_pickle(os.path.join(dic,'NHDPlusData/NHDPlusEROMMA.pk1'))

In [18]:
NHDPlusEROMMA

,NHDPlusID,QAMA,VAMA,QIncrAMA,QBMA,VBMA,QIncrBMA,QCMA,VCMA,QIncrCMA,...,ArQNavMA,PETMA,QLossMA,QGAdjMA,QGNavMA,GageAdjMA,AvgQAdjMA,GageIDMA,GageQMA,VPUID
0,60002600142206,3.814366,0.512549,0.008831,3.814366,0.512549,0.008831,4.966591,0.516961,0.038518,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0405
1,60002600019240,2.790677,0.750890,0.019373,2.790677,0.750890,0.019373,3.597645,0.760088,0.097580,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0405
2,60002600039256,0.000180,0.329112,0.000180,0.000180,0.329112,0.000180,0.000171,0.329022,0.000171,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0405
3,60002600067225,0.064154,0.256144,0.000000,0.064154,0.256144,0.000000,0.073331,0.260759,0.000000,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0405
4,60002600100818,0.043065,0.480599,0.000771,0.043065,0.480599,0.000771,0.048604,0.482295,0.001804,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0405
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118824,60001300118086,0.071466,0.439146,0.071466,0.071466,0.439146,0.071466,0.083045,0.440746,0.083045,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0410
118825,60001300118213,0.119465,0.508180,0.119465,0.119465,0.508180,0.119465,0.138743,0.510615,0.138743,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0410
118826,60001300118258,0.003537,0.350297,0.003537,0.003537,0.350297,0.003537,0.004124,0.350821,0.004124,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0410
118827,60001300118509,0.080878,0.476400,0.080878,0.080878,0.476400,0.080878,0.093970,0.478454,0.093970,...,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0410
